In [78]:
import torch

import pandas as pd
import numpy as np

from tqdm import tqdm
from model import CryptoTransformer, MyDataset
from utils import weighted_corr

In [ ]:
processed_test = pd.read_csv('./processed_data/processed_test.gz')
asset_list = pd.read_csv('./g-research-crypto-forecasting/asset_details.csv')
asset_list.set_index('Asset_ID', inplace=True)
processed_test['Weight'] = processed_test['Asset_ID'].apply(lambda x: asset_list.loc[x]['Weight'])

In [57]:
net = torch.load('./checkpoint/model_v1.0/epoch_3_step_8.pt')
test_dataset = MyDataset(processed_test)

In [58]:
predictions = []
net.eval()
net = net.cpu()
with torch.no_grad():
    with tqdm(test_dataset) as t:
        for x, y, num_assets in t:
            y_pred = net(x)
            y_pred = y_pred[:num_assets]
            predictions.append(y_pred)

100%|██████████| 81010/81010 [01:05<00:00, 1233.35it/s]


In [59]:
predictions = torch.cat(predictions).numpy()
processed_test['Predictions'] = predictions

In [79]:
corr = np.corrcoef(processed_test['Predictions'], processed_test['Target'])[1][0]
w_corr = weighted_corr(processed_test['Predictions'], processed_test['Target'], processed_test['Weight'])
print(f'Pearson Correlation: {corr:.2e}, Weighted Pearson Correlation: {w_corr:.2e}')

Pearson Correlation: -3.52e-03, Weighted Pearson Correlation: -2.33e-03


In [62]:
processed_test['Target']

0         -0.002932
1         -0.002353
2         -0.001510
3          0.001920
4         -0.001293
             ...   
1130823    0.000047
1130824   -0.002006
1130825    0.000281
1130826   -0.001046
1130827    0.000406
Name: Target, Length: 1130828, dtype: float64